In [1]:
# %matplotlib widget

from __future__ import annotations

import re
from collections import defaultdict
from dataclasses import dataclass, field
from itertools import permutations, product
from math import inf
from random import choice

import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import numpy.typing as npt
from mpl_toolkits.mplot3d import axes3d
from numpy import int_, object_
from numpy.typing import NDArray
from test_utilities import run_tests_params
from util import print_hex

COLORS = list(mcolors.CSS4_COLORS.keys())

<link href="style.css" rel="stylesheet"></link>
<article class="day-desc"><h2>--- Day 24: Immune System Simulator 20XX ---</h2><p>After <a href="https://www.youtube.com/watch?v=NDVjLt_QHL8&amp;t=7" target="_blank">a weird buzzing noise</a>, you appear back at the man's cottage. He seems relieved to see his friend, but quickly notices that the little reindeer caught some kind of cold while out exploring.</p>
<p>The portly man explains that this reindeer's immune system isn't similar to regular reindeer immune systems:</p>
<p>The <em>immune system</em> and the <em>infection</em> each have <span title="On second thought, it's pretty similar to regular reindeer immune systems.">an army</span> made up of several <em>groups</em>; each <em>group</em> consists of one or more identical <em>units</em>.  The armies repeatedly <em>fight</em> until only one army has units remaining.</p>
<p><em>Units</em> within a group all have the same <em>hit points</em> (amount of damage a unit can take before it is destroyed), <em>attack damage</em> (the amount of damage each unit deals), an <em>attack type</em>, an <em>initiative</em> (higher initiative units attack first and win ties), and sometimes <em>weaknesses</em> or <em>immunities</em>. Here is an example group:</p>
<pre><code>18 units each with 729 hit points (weak to fire; immune to cold, slashing)
 with an attack that does 8 radiation damage at initiative 10
</code></pre>
<p>Each group also has an <em>effective power</em>: the number of units in that group multiplied by their attack damage. The above group has an effective power of <code>18 * 8 = 144</code>. Groups never have zero or negative units; instead, the group is removed from combat.</p>
<p>Each <em>fight</em> consists of two phases: <em>target selection</em> and <em>attacking</em>.</p>
<p>During the <em>target selection</em> phase, each group attempts to choose one target. In decreasing order of effective power, groups choose their targets; in a tie, the group with the higher initiative chooses first. The attacking group chooses to target the group in the enemy army to which it would deal the most damage (after accounting for weaknesses and immunities, but not accounting for whether the defending group has enough units to actually receive all of that damage).</p>
<p>If an attacking group is considering two defending groups to which it would deal equal damage, it chooses to target the defending group with the largest effective power; if there is still a tie, it chooses the defending group with the highest initiative.  If it cannot deal any defending groups damage, it does not choose a target.  Defending groups can only be chosen as a target by one attacking group.</p>
<p>At the end of the target selection phase, each group has selected zero or one groups to attack, and each group is being attacked by zero or one groups.</p>
<p>During the <em>attacking</em> phase, each group deals damage to the target it selected, if any. Groups attack in decreasing order of initiative, regardless of whether they are part of the infection or the immune system. (If a group contains no units, it cannot attack.)</p>
<p>The damage an attacking group deals to a defending group depends on the attacking group's attack type and the defending group's immunities and weaknesses.  By default, an attacking group would deal damage equal to its <em>effective power</em> to the defending group.  However, if the defending group is <em>immune</em> to the attacking group's attack type, the defending group instead takes <em>no damage</em>; if the defending group is <em>weak</em> to the attacking group's attack type, the defending group instead takes <em>double damage</em>.</p>
<p>The defending group only loses <em>whole units</em> from damage; damage is always dealt in such a way that it kills the most units possible, and any remaining damage to a unit that does not immediately kill it is ignored. For example, if a defending group contains <code>10</code> units with <code>10</code> hit points each and receives <code>75</code> damage, it loses exactly <code>7</code> units and is left with <code>3</code> units at full health.</p>
<p>After the fight is over, if both armies still contain units, a new fight begins; combat only ends once one army has lost all of its units.</p>
<p>For example, consider the following armies:</p>
<pre><code>Immune System:
17 units each with 5390 hit points (weak to radiation, bludgeoning) with
 an attack that does 4507 fire damage at initiative 2
989 units each with 1274 hit points (immune to fire; weak to bludgeoning,
 slashing) with an attack that does 25 slashing damage at initiative 3

Infection:
801 units each with 4706 hit points (weak to radiation) with an attack
that does 116 bludgeoning damage at initiative 1
4485 units each with 2961 hit points (immune to radiation; weak to fire,
cold) with an attack that does 12 slashing damage at initiative 4
</code></pre>

<p>If these armies were to enter combat, the following fights, including details during the target selection and attacking phases, would take place:</p>
<pre><code>Immune System:
Group 1 contains 17 units
Group 2 contains 989 units
Infection:
Group 1 contains 801 units
Group 2 contains 4485 units

Infection group 1 would deal defending group 1 185832 damage
Infection group 1 would deal defending group 2 185832 damage
Infection group 2 would deal defending group 2 107640 damage
Immune System group 1 would deal defending group 1 76619 damage
Immune System group 1 would deal defending group 2 153238 damage
Immune System group 2 would deal defending group 1 24725 damage

Infection group 2 attacks defending group 2, killing 84 units
Immune System group 2 attacks defending group 1, killing 4 units
Immune System group 1 attacks defending group 2, killing 51 units
Infection group 1 attacks defending group 1, killing 17 units
</code></pre>

<pre><code>Immune System:
Group 2 contains 905 units
Infection:
Group 1 contains 797 units
Group 2 contains 4434 units

Infection group 1 would deal defending group 2 184904 damage
Immune System group 2 would deal defending group 1 22625 damage
Immune System group 2 would deal defending group 2 22625 damage

Immune System group 2 attacks defending group 1, killing 4 units
Infection group 1 attacks defending group 2, killing 144 units
</code></pre>
<pre><code>Immune System:
Group 2 contains 761 units
Infection:
Group 1 contains 793 units
Group 2 contains 4434 units

Infection group 1 would deal defending group 2 183976 damage
Immune System group 2 would deal defending group 1 19025 damage
Immune System group 2 would deal defending group 2 19025 damage

Immune System group 2 attacks defending group 1, killing 4 units
Infection group 1 attacks defending group 2, killing 143 units
</code></pre>
<pre><code>Immune System:
Group 2 contains 618 units
Infection:
Group 1 contains 789 units
Group 2 contains 4434 units

Infection group 1 would deal defending group 2 183048 damage
Immune System group 2 would deal defending group 1 15450 damage
Immune System group 2 would deal defending group 2 15450 damage

Immune System group 2 attacks defending group 1, killing 3 units
Infection group 1 attacks defending group 2, killing 143 units
</code></pre>
<pre><code>Immune System:
Group 2 contains 475 units
Infection:
Group 1 contains 786 units
Group 2 contains 4434 units

Infection group 1 would deal defending group 2 182352 damage
Immune System group 2 would deal defending group 1 11875 damage
Immune System group 2 would deal defending group 2 11875 damage

Immune System group 2 attacks defending group 1, killing 2 units
Infection group 1 attacks defending group 2, killing 142 units
</code></pre>
<pre><code>Immune System:
Group 2 contains 333 units
Infection:
Group 1 contains 784 units
Group 2 contains 4434 units

Infection group 1 would deal defending group 2 181888 damage
Immune System group 2 would deal defending group 1 8325 damage
Immune System group 2 would deal defending group 2 8325 damage

Immune System group 2 attacks defending group 1, killing 1 unit
Infection group 1 attacks defending group 2, killing 142 units
</code></pre>
<pre><code>Immune System:
Group 2 contains 191 units
Infection:
Group 1 contains 783 units
Group 2 contains 4434 units

Infection group 1 would deal defending group 2 181656 damage
Immune System group 2 would deal defending group 1 4775 damage
Immune System group 2 would deal defending group 2 4775 damage

Immune System group 2 attacks defending group 1, killing 1 unit
Infection group 1 attacks defending group 2, killing 142 units
</code></pre>
<pre><code>Immune System:
Group 2 contains 49 units
Infection:
Group 1 contains 782 units
Group 2 contains 4434 units

Infection group 1 would deal defending group 2 181424 damage
Immune System group 2 would deal defending group 1 1225 damage
Immune System group 2 would deal defending group 2 1225 damage

Immune System group 2 attacks defending group 1, killing 0 units
Infection group 1 attacks defending group 2, killing 49 units
</code></pre>
<pre><code>Immune System:
No groups remain.
Infection:
Group 1 contains 782 units
Group 2 contains 4434 units
</code></pre>
<p>In the example above, the winning army ends up with <code>782 + 4434 = <em>5216</em></code> units.</p>
<p>You scan the reindeer's condition (your puzzle input); the white-bearded man looks nervous.  As it stands now, <em>how many units would the winning army have</em>?</p>
</article>


In [86]:
from enum import Enum, auto
from re import findall, split, compile

from more_itertools import one


example = """
Immune System:
17 units each with 5390 hit points (weak to radiation, bludgeoning) with an attack that does 4507 fire damage at initiative 2
989 units each with 1274 hit points (immune to fire; weak to bludgeoning, slashing) with an attack that does 25 slashing damage at initiative 3

Infection:
801 units each with 4706 hit points (weak to radiation) with an attack  that does 116 bludgeoning damage at initiative 1
4485 units each with 2961 hit points (immune to radiation; weak to fire, cold) with an attack that does 12 slashing damage at initiative 4
"""


class Enumeration(Enum):
    def __repr__(self):
        return f"{self.name}"


class AttackType(Enumeration):
    bludgeoning = auto()
    cold = auto()
    fire = auto()
    radiation = auto()
    slashing = auto()


class Immunity(Enumeration):
    bludgeoning = auto()
    cold = auto()
    fire = auto()
    radiation = auto()
    slashing = auto()


class Weakness(Enumeration):
    bludgeoning = auto()
    cold = auto()
    fire = auto()
    radiation = auto()
    slashing = auto()


@dataclass
class Group:
    units: int
    hit_points: int  # (amount of damage a unit can take before it is destroyed)
    attack_damage: int  # (the amount of damage each unit deals)
    attack_type: AttackType
    initiative: int
    immunities: list[Immunity]
    weaknesses: list[Weakness]

    @classmethod
    def parse(cls, s: str) -> Group:
        attack_regex = compile(rf'\d+\s({r"|".join(AttackType._member_names_)})')

        units, hit_points, attack_damage, initiative = (
            int(i) for i in findall(r"-?\d+", s)
        )
        attack_type = one(findall(attack_regex, s))
        weaknesses, immunities = [], []
        for o in findall(r"\(.*?\)", s):
            for oo in o[1:-1].split("; "):
                if oo.startswith("weak"):
                    weaknesses = {
                        Weakness[w] for w in oo.removeprefix("weak to ").split(", ")
                    }
                else:
                    immunities = {
                        Immunity[w] for w in oo.removeprefix("immune to ").split(", ")
                    }

        return Group(
            units=units,
            hit_points=hit_points,
            attack_damage=attack_damage,
            attack_type=attack_type,
            initiative=initiative,
            immunities=immunities,
            weaknesses=weaknesses,
        )


class System:
    def __init__(self, immune: list[Group], infection: list[Group]) -> None:
        self.immune = immune
        self.infection = infection

    def __repr__(self) -> str:
        return "\n".join(
            [
                "Immune System:",
                "\n".join(
                    f"Group {c} contains {im.units} units"
                    for c, im in enumerate(self.immune, start=1)
                ),
                "Infection:",
                "\n".join(
                    f"Group {c} contains {im.units} units"
                    for c, im in enumerate(self.infection, start=1)
                ),
            ]
        )

    @classmethod
    def parse(cls, s: str) -> System:
        immune, infection = split(r"\n\s*\n", s.strip())
        immune_groups = [Group.parse(ig) for ig in immune.splitlines()[1:]]
        infection_groups = [Group.parse(ig) for ig in infection.splitlines()[1:]]
        system = System(immune_groups, infection_groups)
        return system


System.parse(example)

Immune System:
Group 1 contains 17 units
Group 2 contains 989 units
Infection:
Group 1 contains 801 units
Group 2 contains 4485 units

In [85]:
with open("../input/day24.txt") as f:
    puzzle = f.read()

System.parse(puzzle)

Immune System:
Group 1 contains 3400 units
Group 2 contains 138 units
Group 3 contains 255 units
Group 4 contains 4145 units
Group 5 contains 3605 units
Group 6 contains 865 units
Group 7 contains 633 units
Group 8 contains 2347 units
Group 9 contains 7045 units
Group 10 contains 1086 units
Infection:
Group 1 contains 2152 units
Group 2 contains 40 units
Group 3 contains 59 units
Group 4 contains 1569 units
Group 5 contains 929 units
Group 6 contains 5264 units
Group 7 contains 1570 units
Group 8 contains 1428 units
Group 9 contains 1014 units
Group 10 contains 7933 units

<link href="style.css" rel="stylesheet"></link>
...


<link href="style.css" rel="stylesheet"></link>
...
